In [ ]:
# 创建数据库
from pony.orm import *
db = Database()
class Tm(db.Entity):
    id = PrimaryKey(int, auto=True)
    bh = Optional(str)
    bt = Optional(str)
    fs = Optional(int)
    tgs = Optional(int)
    tjs = Optional(int)
    tgl = Optional(str)
    tmnl = Optional(str)
    html = Optional(str)
    tx = Optional(str)
    xqylj = Optional(str)


In [ ]:
# 生成数据库
def create_database():
    db = Database()
    dbpath = 'e:/ex8/tm.sqlite'
    if os.path.exists(dbpath):
        os.remove(dbpath)
    f = open(dbpath,"w")
    f.close()
    db.bind(provider='sqlite', filename=r'e:/ex8/tm.sqlite')

    db.generate_mapping(create_tables=True)

    set_sql_debug(True)

In [ ]:
# 获取网页源代码
import time
def _get_html(url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(1)
    html = driver.page_source
    return html
# 取两个网址
def _get_url():
    urls = ['https://pintia.cn/problem-sets/1111652100718116864/problems/type/7','https://pintia.cn/problem-sets/1111652100718116864/problems/type/6']
    return urls

In [ ]:
# 取标号
def _get_bh(html):
    bhs = []
    reobj = re.compile(r'<tr><td class="answerIcon_1du7d"></td><td>([\d\D]*?)</td><td><a href=[\d\D]*?')
    for match in reobj.finditer(html):
        bhs.append(match.group(1))
    return bhs

In [ ]:
# 取题型
def _get_tx(html):
    match = re.search(r'class="nav-link active">[\d\D]*?class="pc-text-raw">([\d\D]*?)</div></div><span', html)
    if match:
        txs = match.group(1)
    return txs

In [ ]:
# 取标题
def _get_bt(html):
    bts = []
    reobj = re.compile(r'<td><a href="[\d\D]*?class="">([\d\D]*?)</a></td>')
    for match in reobj.finditer(html):
        bts.append(match.group(1))
    return bts

In [ ]:
# 取分数
def _get_fs(html):
    fss = []
    reobj = re.compile(r'class="answerIcon_1du7d"></td><td>[\d\D]*?href="[\d\D]*?</a></td><td>([\d\D]*?)</td><td>')
    for match in reobj.finditer(html):
        fss.append(match.group(1))
    return fss


In [ ]:
#取 通过数
def _get_tgs(html):
    tgss = []
    reobj = re.compile(r'class="answerIcon_1du7d"></td><td>[\d\D]*?href="[\d\D]*?</a></td><td>[\d\D]*?</td><td>([\d\D]*?)</td><td>')
    for match in reobj.finditer(html):
        tgss.append(match.group(1))
    return tgss

In [ ]:
# def _get_tmnl(html):
    '''爬取该网页题目内容'''
    html3 = etree.HTML(html,etree.HTMLParser())
    tmnl = html3.xpath('//*[@id="sparkling-daydream"]/div[3]/div[3]/div/div[4]/div[2]/div[1]/div/p[1]//text()')
    return tmnl

def task4():
    '''将爬取的题目内容入库'''
    with db_session:
        for s in Tm.select():
            html = s.html
            if _get_tmnl(html) != []:
                s.tmnl = _get_tmnl(s.html)[0]
                db.commit()取提交数
def _get_tjs(html):
    tjss = []
    reobj = re.compile(r'href="[\d\D]*?</a></td><td>[\d\D]*?</td><td>[\d\D]*?</td><td>([\d\D]*?)</td><td>')
    for match in reobj.finditer(html):
        tjss.append(match.group(1))
    return tjss

In [ ]:
# 取通过率
def _get_tgl(html):
    tgls = []
    reobj = re.compile(r'href="[\d\D]*?</a></td><td>[\d\D]*?</td><td>[\d\D]*?</td><td>[\d\D]*?</td><td>([\d\D]*?)</td></tr>')
    for match in reobj.finditer(html):
        tgls.append(match.group(1))
    return tgls

In [ ]:
# 详情页链接
def _get_xqylj(html):
    xqyljs = []
    reobj = re.compile(r'<td><a href="([\d\D]*?)" class="">')
    for match in reobj.finditer(html):
        xqyljs.append(match.group(1))
    return xqyljs

In [ ]:
# 信息入库
@db_session
def task1():
    for url in _get_url():
        html = _get_html(url)
        for i in range(len(_get_bh(html))):
            s = Tm(bh = _get_bh(html)[i],tx = _get_tx(html),bt = _get_bt(html)[i],fs = _get_fs(html)[i],tgs = _get_tgs(html)[i],tjs = _get_tjs(html)[i],tgl = _get_tgl(html)[i],xqylj = _get_xqylj(html)[i])
            db.commit()

In [ ]:
# 题目html入库
def _get_html2(href):
    driver = webdriver.Chrome()
    url = 'http://pintia.cn' + href
    driver.get(url)
    time.sleep(5)
    html2 = driver.page_source
    driver.quit()
    return html2

def task2():
    with db_session:
        for s in Tm.select():
            s.html = _get_html2(s.xqylj)
            db.commit()

In [ ]:
# 题目内容入库
def _get_tmnl(html):
    html3 = etree.HTML(html,etree.HTMLParser())
    tmnl = html3.xpath('//*[@id="sparkling-daydream"]/div[3]/div[3]/div/div[4]/div[2]/div[1]/div/p[1]//text()')
    return tmnl

def task3():
    with db_session:
        for s in Tm.select():
            html = s.html
            if _get_tmnl(html) != []:
                s.tmnl = _get_tmnl(s.html)[0]
                db.commit()